In [826]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [827]:
df = pd.read_csv('kickstarter_data_with_features.csv')

In [828]:
#These are the only columns we can use as input
df = df[['name', 'goal', 'blurb', 'launched_at', 'deadline','category','state', 'country']]

In [829]:
#filter out all countries not english-speaking
english_countries = ['US', 'IE', 'GB', 'AU', 'CA', 'NZ', ]
df= df[df['country'].isin(english_countries)]

In [830]:
# df = pd.read_csv('ks-projects-201801.csv')

In [831]:
#create 'success'filter which only shows failed or success and cuts out other options
suc_filt = ['failed', 'successful', 'canceled']
df = df[df['state'].isin(suc_filt)]
# Convert fail or succuess to 0 or 1
df['state'] = df['state'].replace({'failed': 0, 'successful': 1, 'canceled': 0})

In [832]:
#Two seperate dataframes are created below, one of only success and one of only fail
faileddf = df[df['state'] == 0]
sucdf = df[df['state'] == 1]

In [833]:
#Baseline prediction
failed = len(faileddf)
suc = len(sucdf)

suc / failed

0.4579348354676609

In [834]:
columns = ['name','blurb', 'state']

to_df = df.copy()

to_df = to_df[columns]

In [835]:
to_df.fillna(' ', inplace=True)

In [836]:
to_df['text']=to_df['name']+' '+to_df['blurb']

In [837]:
df_nt = df[['goal', 'launched_at', 'deadline', 'country', 'state']]

In [838]:
#convert time columns to campaign length
df['deadline'] = df['deadline'].str[:10]
df['launched_at'] = df['launched_at'].str[:10]
df_nt['deadline'] = pd.to_datetime(df_nt['deadline'], format="%Y/%m/%d")
df_nt['launched_at'] = pd.to_datetime(df_nt['launched_at'], format="%Y/%m/%d")

df['deadline'] = pd.to_datetime(df['deadline'], format="%Y/%m/%d")
df['launched_at'] = pd.to_datetime(df['launched_at'], format="%Y/%m/%d")
#create new column
df_nt['cam_length'] = df_nt['deadline'] - df_nt['launched_at']
df['cam_length'] = df['deadline'] - df['launched_at']

In [839]:
df

,name,goal,blurb,launched_at,deadline,category,state,country,cam_length
0,Auntie Di's Music Time Sign ASL for Hearing an...,1500.0,MTS ASL Curriculum Workbook is a reproducible ...,2014-12-17,2015-01-23,Academic,0,US,37 days
1,Jump Start Kindergarten Toolkit,500.0,"This kit teaches how to print, correct an ugly...",2015-03-02,2015-05-01,Academic,0,US,60 days
2,Ojukwu Balewa Awolowo (O.B.A.) Public Library ...,100000.0,"Establishing a free, world-class, public libra...",2015-01-25,2015-03-26,Academic,0,US,60 days
3,"MASTIZE - [mas-TAHYZ, MAS-tahyz] - to spread",5000.0,Goal: Introducing a new word into the English ...,2014-09-06,2014-10-06,Academic,0,US,30 days
5,Shadow School Board - Reforming Texas School B...,13000.0,Shadow School Board will provide parents & tax...,2015-10-21,2015-11-20,Academic,0,US,30 days
...,...,...,...,...,...,...,...,...,...
20625,Private.Center - Private Social Network,20000.0,Private.Center - the Private Social Network is...,2015-12-17,2016-02-15,Apps,0,GB,60 days
20626,Pathfinder Adventure Card Game Character Track...,5000.0,A companion app for the Pathfinder Adventure C...,2016-05-26,2016-07-01,Apps,0,AU,36 days
20629,Sloth Face App,10000.0,"We all love fatbooth and Ugly booth, why not s...",2015-02-13,2015-04-14,Apps,0,US,60 days
20630,DriverTools,2500.0,Production android app for cab drivers and tru...,2015-04-20,2015-05-20,Apps,0,US,30 days


In [840]:
lb_make = LabelEncoder()
df["cam_length"] = lb_make.fit_transform(df["cam_length"]) + 1

In [841]:
df_nt = df_nt[['goal', 'cam_length', 'state']]

In [842]:
df

,name,goal,blurb,launched_at,deadline,category,state,country,cam_length
0,Auntie Di's Music Time Sign ASL for Hearing an...,1500.0,MTS ASL Curriculum Workbook is a reproducible ...,2014-12-17,2015-01-23,Academic,0,US,37
1,Jump Start Kindergarten Toolkit,500.0,"This kit teaches how to print, correct an ugly...",2015-03-02,2015-05-01,Academic,0,US,60
2,Ojukwu Balewa Awolowo (O.B.A.) Public Library ...,100000.0,"Establishing a free, world-class, public libra...",2015-01-25,2015-03-26,Academic,0,US,60
3,"MASTIZE - [mas-TAHYZ, MAS-tahyz] - to spread",5000.0,Goal: Introducing a new word into the English ...,2014-09-06,2014-10-06,Academic,0,US,30
5,Shadow School Board - Reforming Texas School B...,13000.0,Shadow School Board will provide parents & tax...,2015-10-21,2015-11-20,Academic,0,US,30
...,...,...,...,...,...,...,...,...,...
20625,Private.Center - Private Social Network,20000.0,Private.Center - the Private Social Network is...,2015-12-17,2016-02-15,Apps,0,GB,60
20626,Pathfinder Adventure Card Game Character Track...,5000.0,A companion app for the Pathfinder Adventure C...,2016-05-26,2016-07-01,Apps,0,AU,36
20629,Sloth Face App,10000.0,"We all love fatbooth and Ugly booth, why not s...",2015-02-13,2015-04-14,Apps,0,US,60
20630,DriverTools,2500.0,Production android app for cab drivers and tru...,2015-04-20,2015-05-20,Apps,0,US,30


In [843]:
df = df[['name', 'goal', 'blurb', 'state', 'cam_length']]
NUMERIC_COLUMNS = ['goal', "cam_length"]
LABELS = ['state']
numeric_data_only = df[NUMERIC_COLUMNS]
NON_LABELS = [c for c in df.columns if c not in LABELS]
TEXT_COLUMNS = [c for c in NON_LABELS if c not in NUMERIC_COLUMNS]

In [844]:
TEXT_COLUMNS

['name', 'blurb']

In [845]:
def declare(df):
    NUMERIC_COLUMNS = ['goal', "cam_length"]
    LABELS = ['state']
    numeric_data_only = df[NUMERIC_COLUMNS]
    NON_LABELS = [c for c in df.columns if c not in LABELS]
    TEXT_COLUMNS = [c for c in NON_LABELS if c not in NUMERIC_COLUMNS]

    return df

In [846]:
declare(df)

,name,goal,blurb,state,cam_length
0,Auntie Di's Music Time Sign ASL for Hearing an...,1500.0,MTS ASL Curriculum Workbook is a reproducible ...,0,37
1,Jump Start Kindergarten Toolkit,500.0,"This kit teaches how to print, correct an ugly...",0,60
2,Ojukwu Balewa Awolowo (O.B.A.) Public Library ...,100000.0,"Establishing a free, world-class, public libra...",0,60
3,"MASTIZE - [mas-TAHYZ, MAS-tahyz] - to spread",5000.0,Goal: Introducing a new word into the English ...,0,30
5,Shadow School Board - Reforming Texas School B...,13000.0,Shadow School Board will provide parents & tax...,0,30
...,...,...,...,...,...
20625,Private.Center - Private Social Network,20000.0,Private.Center - the Private Social Network is...,0,60
20626,Pathfinder Adventure Card Game Character Track...,5000.0,A companion app for the Pathfinder Adventure C...,0,36
20629,Sloth Face App,10000.0,"We all love fatbooth and Ugly booth, why not s...",0,60
20630,DriverTools,2500.0,Production android app for cab drivers and tru...,0,30


In [847]:
def combine_text_columns(data_frame, to_drop=['state', 'goal', 'cam_length']):
    """ Takes the dataset as read in, drops the non-feature, non-text columns and
        then combines all of the text columns into a single vector that has all of
        the text for a row.

        :param data_frame: The data as read in with read_csv (no preprocessing necessary)
        :param to_drop (optional): Removes the numeric and label columns by default.
    """
    # drop non-text columns that are in the df
    # to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = set(data_frame.drop(to_drop, axis=1)) & set(data_frame.columns.tolist())
    text_data = data_frame['name'] + " " + data_frame['blurb']

    # joins all of the text items in a row (axis=1)
    # with a space in between
    return text_data

In [848]:
combine_text_columns(df)

0        Auntie Di's Music Time Sign ASL for Hearing an...
1        Jump Start Kindergarten Toolkit This kit teach...
2        Ojukwu Balewa Awolowo (O.B.A.) Public Library ...
3        MASTIZE - [mas-TAHYZ, MAS-tahyz]  - to spread ...
5        Shadow School Board - Reforming Texas School B...
                               ...                        
20625    Private.Center - Private Social Network Privat...
20626    Pathfinder Adventure Card Game Character Track...
20629    Sloth Face App We all love fatbooth and Ugly b...
20630    DriverTools Production android app for cab dri...
20631    Let's Trail app We recently launched version 1...
Length: 17988, dtype: object

In [849]:
def get_numeric(df):
    data_frame = df[['goal', 'cam_length']]
    return data_frame

In [852]:
df['text'] = combine_text_columns(df)

In [853]:
df = df[['goal', 'state', 'cam_length', 'text']]

In [854]:
df

,goal,state,cam_length,text
0,1500.0,0,37,Auntie Di's Music Time Sign ASL for Hearing an...
1,500.0,0,60,Jump Start Kindergarten Toolkit This kit teach...
2,100000.0,0,60,Ojukwu Balewa Awolowo (O.B.A.) Public Library ...
3,5000.0,0,30,"MASTIZE - [mas-TAHYZ, MAS-tahyz] - to spread ..."
5,13000.0,0,30,Shadow School Board - Reforming Texas School B...
...,...,...,...,...
20625,20000.0,0,60,Private.Center - Private Social Network Privat...
20626,5000.0,0,36,Pathfinder Adventure Card Game Character Track...
20629,10000.0,0,60,Sloth Face App We all love fatbooth and Ugly b...
20630,2500.0,0,30,DriverTools Production android app for cab dri...


In [855]:
# df['text'] = df['text'].tolist()

In [856]:
df['text']

0        Auntie Di's Music Time Sign ASL for Hearing an...
1        Jump Start Kindergarten Toolkit This kit teach...
2        Ojukwu Balewa Awolowo (O.B.A.) Public Library ...
3        MASTIZE - [mas-TAHYZ, MAS-tahyz]  - to spread ...
5        Shadow School Board - Reforming Texas School B...
                               ...                        
20625    Private.Center - Private Social Network Privat...
20626    Pathfinder Adventure Card Game Character Track...
20629    Sloth Face App We all love fatbooth and Ugly b...
20630    DriverTools Production android app for cab dri...
20631    Let's Trail app We recently launched version 1...
Name: text, Length: 17988, dtype: object

In [857]:
from sklearn.feature_extraction.text import CountVectorizer
import re

In [858]:
df.isnull().sum()

goal          0
state         0
cam_length    0
text          5
dtype: int64

In [859]:
df.dropna(inplace = True)
df.reset_index(drop=True, inplace=True)

In [860]:
# new_doc = [df['text'][0]]

In [861]:
# new_doc

In [862]:
# X = vectorizer.fit_transform(new_doc)

In [863]:
# x = X.toarray()

In [864]:
# x = x.ravel()

In [865]:
# x.shape

In [866]:
df['text'][17674]

'O ---'

In [867]:
def tokenize(text):    
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

In [868]:
vectorizer = CountVectorizer(stop_words='english', tokenizer=tokenize,ngram_range=(1,3))

In [869]:
num = 0
for i in df['text']:
    new_doc = []
    new_doc = [df['text'][num]]
    X = vectorizer.fit_transform(new_doc)
    x = X.toarray()
    x = x.ravel()
    df['text'][num] = x
    num += 1

In [870]:
df['text'].shape

(17983,)

In [871]:
# num = 0
# for i in df['text']:
#     df['text'][num] = df['text'][num].tolist()
#     num += 1

In [872]:
num = 0
for i in df['text']:
    # df['text'][num] = df['text'][num].tolist()
    df['text'][num] = df['text'][num].sum()
    num += 1

In [873]:
df

,goal,state,cam_length,text
0,1500.0,0,37,72
1,500.0,0,60,51
2,100000.0,0,60,48
3,5000.0,0,30,42
4,13000.0,0,30,60
...,...,...,...,...
17978,20000.0,0,60,45
17979,5000.0,0,36,60
17980,10000.0,0,60,48
17981,2500.0,0,30,18


In [874]:
# split into train test
X_train, X_test, y_train, y_test = train_test_split(df[['goal', 'cam_length']], df['state'],train_size=0.80, test_size=0.20, stratify= df['state'], random_state=42)

In [875]:
pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    XGBClassifier()
)

In [876]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [877]:
pipe_pred = pipe.predict(X_test)

In [878]:
accuracy_score(pipe_pred,y_test)

0.705031971087017

In [25]:
%%time
pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    RandomForestClassifier(
        n_jobs=1,
        random_state=42,
        n_estimators=9,
    )
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__criterion': ('gini', 'entropy'),
    'randomforestclassifier__max_depth': (8,10,12,14,15,16,20,25),
    'randomforestclassifier__min_samples_split': (2,4,6,8,10)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=240,
    cv=7,
    scoring='accuracy',
    verbose=1,
    return_train_score=True,
    n_jobs=-1,
)

search.fit(X_train,y_train)

print('Best Score:',search.best_score_)
print('Best param:',search.best_params_)
print('Best estimaator:', search.best_estimator_)

Fitting 7 folds for each of 160 candidates, totalling 1120 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   13.7s
Best Score: 0.702570805083657
Best param: {'simpleimputer__strategy': 'mean', 'randomforestclassifier__min_samples_split': 8, 'randomforestclassifier__max_depth': 8, 'randomforestclassifier__criterion': 'entropy'}
Best estimaator: Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=8,
                                        min_samples_split=8, n_estimators=9,
                                        n_jobs=1, random_state=42))])
Wall time:

In [879]:
%%time
pipe2 = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    RandomForestClassifier(criterion='entropy', max_depth=8,
                                        min_samples_split=8, n_estimators=999,
                                        n_jobs=-1, random_state=42)
    )

pipe2.fit(X_train,y_train)

Wall time: 1.85 s


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=8,
                                        min_samples_split=8, n_estimators=999,
                                        n_jobs=-1, random_state=42))])

In [880]:
pipe2_pred = pipe2.predict(X_test)

In [881]:
accuracy_score(pipe2_pred,y_test)

0.7108701695857659

In [882]:
import pickle

In [883]:
dill = pickle.dump(pipe2, open( "kickstarter1.pkl", "wb" ) )

In [31]:
file_pickle = open("kickstarter.pkl",'rb')

In [32]:
clf2 = pickle.load(file_pickle)

In [33]:
clf_pred = clf2.predict(X_test)

In [34]:
accuracy_score(clf_pred,y_test)

0.717065036131184